In [ ]:
!pip install torch torchvision opencv-python matplotlib imutils
!pip install timm torch torchvision opencv-python matplotlib

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from PIL import Image
import imutils
import timm  # EfficientNet is available through the timm library
from torchvision.datasets import ImageFolder
from google.colab import drive
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
import torchvision.models as models
import random
from torchvision import transforms
from sklearn.metrics import classification_report

In [ ]:
# %% [code]
%run /content/Brain-Tumor-Images-Classification/Data-Preprocessing/Brain_tumor_data_preprocessing.ipynb

In [ ]:
# %% [code]
%run /content/Brain-Tumor-Images-Classification/Utils/Save_Load__Model.ipynb

In [ ]:

# Load pre-trained VGG16 model
model = models.vgg16(pretrained=True)

# Modify the final fully connected layer for your number of classes
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, len(dataset.classes))

# Set device to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Freeze all layers except the final classifier
for param in model.parameters():
    param.requires_grad = False
# Unfreeze the final classifier layer
for param in model.classifier[6].parameters():
    param.requires_grad = True

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()

# Pass only the parameters of the final layer to the optimizer
optimizer = optim.Adam(model.classifier[6].parameters(), lr=0.0001)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {running_loss/len(train_loader):.4f}")
    # Save checkpoint after each epoch
    checkpoint = {
       'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': running_loss / len(train_loader)
    }
    checkpoint_path = "/content/drive/My Drive/vgg16_brain_tumor_checkpoint.pth"
    save_checkpoint(checkpoint, checkpoint_path)

print("Training complete!")

# Evaluation
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy: {accuracy * 100:.2f}%")


In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
from torchvision import transforms

# Function to test a random image
def test_random_image(model, test_loader, device, class_names):
    model.eval()  # Set the model to evaluation mode

    # Get a random sample from the test_loader
    dataiter = iter(test_loader)
    images, labels = next(dataiter)

    # Select a random image and its corresponding label
    idx = random.randint(0, len(images) - 1)
    img = images[idx].unsqueeze(0).to(device)  # Add batch dimension and move to device
    true_label = labels[idx].item()

    # Make prediction
    with torch.no_grad():  # Disable gradient calculation
        output = model(img)
        _, predicted = torch.max(output, 1)

    predicted_label = predicted.item()

    # Convert the image back to a format for display
    img = img.squeeze().cpu().numpy().transpose(1, 2, 0)  # Convert to HWC format

    # Display the image along with predicted and actual labels
    plt.imshow(img)
    plt.title(f"Predicted: {class_names[predicted_label]} | Actual: {class_names[true_label]}")
    plt.axis('off')
    plt.show()

    # Print the actual and predicted labels
    print(f"Predicted Label: {class_names[predicted_label]}")
    print(f"Actual Label: {class_names[true_label]}")

In [ ]:
# Call the function to test on a random image
test_random_image(model, test_loader, device, class_names)